In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload

In [3]:
import drivers, lib
from time import sleep
import pickle

In [4]:
from lib import data_management as dm

In [5]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
from drivers.Agilent_PNA_L import *
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.E8257D import MXG, EXG

In [6]:
exa = Agilent_EXA_N9010A("EXA")
ro_awg = KeysightAWG("AWG3")
q_awg = KeysightAWG("AWG2")
ro_lo = EXG("EXG")

In [38]:
q_lo = MXG("MXG")

In [19]:
ro_lo = Agilent_PNA_L("PNA-L2")
ro_lo.set_frequency = ro_lo.set_center
ro_lo.set_output_state = lambda x: x
ro_lo.set_span(0)
ro_lo.set_nop(1)
ro_lo.sweep_continuous()

In [31]:
reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

##  Readout resonator

In [28]:
ro_resonator_frequency = 7.047e9
if_offset = 0

In [33]:
cal = IQCalibrator(ro_awg, exa, ro_lo, "TEST", -20)
res = cal.calibrate(lo_frequency=ro_resonator_frequency, if_frequency=if_offset, lo_power=0,
                    ssb_power=-30, waveform_resolution=1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=500,)
#                     initial_guess=res.get_optimization_results()[0])
print(res)

DC offsets open:  [ 1.872 1.872 ] [ -29.999 ], Calibration data for mixer TEST
Mixer parameters: {'mixer_id': 'TEST', 'iq_attenuation': -20}
Radiation parameters: {'lo_frequency': 7047000000.0, 'lo_power': 0, 'if_frequency': 0, 'ssb_power': -30, 'waveform_resolution': 1}
Optimization results: {'dc': -104.71302795410156, 'dc_open': [-29.99901008605957]}
Optimization parameters {'dc_offsets': array([ 0.05537704, -0.0996438 ]), 'dc_offsets_open': array([ 1.87090006,  1.87089044]), 'if_offsets': None, 'if_amplitudes': None, 'if_phase': None}
Optimization time: 0 h 0 m 27.74 s
Finished at: 2017-04-25 19:51:01.060578


In [35]:
dm.save_IQMX_calibration(res)

In [7]:
calibrations=dm.load_IQMX_calibration_database("TEST", -20)
res = calibrations.get(frozenset(dict(lo_power=10, ssb_power=-30, lo_frequency=7e9, 
                                      if_frequency=10e6, waveform_resolution=0.1).items()))

## Qubit

In [36]:
qubit_frequency = 7.9e9
if_frequency = 50e6

In [39]:
cal = IQCalibrator(q_awg, exa, q_lo, "TESTQ", -6)
res = cal.calibrate(lo_frequency=qubit_frequency+if_frequency, if_frequency=if_frequency, lo_power=15,
                    ssb_power=-10, waveform_resolution=.1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=100,)
#                     initial_guess=res.get_optimization_results()[0])
print(res)

Phase:  0.4804 [ -10.124 -65.914 -90.034 ]              ]          Calibration data for mixer TESTQ
Mixer parameters: {'mixer_id': 'TESTQ', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 7950000000.0, 'lo_power': 15, 'if_frequency': 50000000.0, 'ssb_power': -10, 'waveform_resolution': 0.1}
Optimization results: {'dc': -82.644966125488281, 'if': [-10.123542785644531, -65.9144287109375, -90.03401947021484]}
Optimization parameters {'dc_offsets': array([ 0.04466626, -0.07696024]), 'dc_offsets_open': None, 'if_offsets': array([ 0.04724813, -0.07887387]), 'if_amplitudes': array([-0.8938785 ,  0.87966588]), 'if_phase': array([ 1.50910594])}
Optimization time: 0 h 1 m 29.14 s
Finished at: 2017-04-25 22:59:24.722111


In [40]:
dm.save_IQMX_calibration(res)

In [41]:
q_lo.visa_instr.close()

In [57]:
q_lo.set_power(15)

In [75]:
calibrations = (dm.load_IQMX_calibration_database("TESTQ", -20))
cal = calibrations.get(frozenset(dict(lo_power=15, ssb_power=-40, lo_frequency=qubit_frequency+if_frequency, 
                                      if_frequency=if_frequency, waveform_resolution=0.1).items()))

In [76]:
print(cal)

Calibration data for mixer TESTQ
Mixer parameters: {'mixer_id': 'TESTQ', 'iq_attenuation': -20}
Radiation parameters: {'lo_frequency': 9250000000.0, 'lo_power': 15, 'if_frequency': 50000000.0, 'ssb_power': -40, 'waveform_resolution': 0.1}
Optimization results: {'dc': -111.25095367431641, 'if': [-39.540977478027344, -83.05058288574219, -98.06412506103516]}
Optimization parameters {'dc_offsets': array([-0.4119494 ,  0.51782169]), 'if_offsets': array([-0.48734834,  0.68734449]), 'if_amplitudes': array([ 5.17142139,  5.58796915]), 'if_phase': array([ 1.6731673])}
Optimization time: 0 h 1 m 26.33 s
Finished at: 2017-03-27 21:10:28.338740


In [30]:
q_lo.visa_instr.close()